In [18]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import pandas as pd


## scraping movies titles from wikipedia

In [204]:
domain="https://en.wikipedia.org/"
page = requests.get("https://en.wikipedia.org/wiki/Lists_of_American_films")
soup = BeautifulSoup(page.content, 'html.parser')

In [205]:
list_by_years=[]
for part in soup.select('li'):
    for link in part.select('a'):
        strlink=str(link.get('title'))
        try:
            if(strlink.startswith('List of American')and int(strlink.split()[-1])>=2016 ):
                list_by_years.append(link.get('href'))
        except:
            pass     

In [207]:

title=[]
for m,link in enumerate(list_by_years):
    page=requests.get(domain+link)
    
    soup=BeautifulSoup(page.content, 'html.parser')
    tables=soup.select('table')
 
    print('pass',m)
    for table in tables:
        if('wikitable' in str(table.get('class'))):
            if('Cast' in str(table.find('tr'))):
            
                temp=table.select('tr')
                for i in temp[1:]:
                    aim=[]
                    for j in i.select('td'):
                        if(len(j.select('a'))!=0):
                            a=''
                            for k in j.select('a'):
                                a+=k.getText()+','
                            aim.append(a[:-1])
                        else:
                            aim.append(j.getText())
                    e=0
                    if(aim[0][:-1].isdigit()):
                        e=1
                    title.append(aim[e])

pass 0
pass 1
pass 2
pass 3
pass 4
pass 5
pass 6
pass 7


get ids tmdb of these movies

In [22]:
from urllib.request import urlopen
import json

In [ ]:
api_key="your api key ,you can get it from tmdb"

info_films=[]
for l,k in enumerate(title):
    print(l)
    url ="https://api.themoviedb.org/3/search/movie?api_key="+api_key+"&query='"+str(k)+"'"
    url=url.replace("\n", " ").encode('ascii', 'ignore').decode('ascii')
    a=url.replace(" ", "%20").encode('ascii', 'ignore').decode('ascii')
    response = urlopen(a)
    data_json = json.loads(response.read())
    for i in data_json['results']:
        info_films.append(i)




In [217]:

data=pd.DataFrame(info_films)

In [223]:
data['release_date']=pd.to_datetime(data['release_date'])
data=data[~(data.release_date<'2016-01-01')]


In [ ]:
data=data.drop_duplicates(subset ="id")

In [258]:
id=pd.DataFrame(id)
id.rename(columns = {'tmdb_id':'id'}, inplace = True)

In [259]:
id

,id,imdb_id
0,329440,tt3387542
1,334538,tt2625810
2,929562,tt13924926
3,857837,tt11718696
4,634571,tt11023922
...,...,...
5113,474876,tt0887908
5114,787699,tt6166392
5115,558915,tt1200263
5116,797898,None


In [ ]:
data_kaggle=pd.read_csv('movies_metadata.csv')

Now, we will get ids of  45.000 movies from Kaggle data, and concatenate those ids with others

In [ ]:
ids=pd.concat([data_kaggle['id'],data['id']])
ln=len(ids)

So, we have  more than 50.000 movies in our dataset.

In [19]:
len(ids)

50145

## load movies details from tmdb

collect detail for each movie

In [23]:

info=[]
errors=[]
for i in ids:
    try:    
        url ="https://api.themoviedb.org/3/movie/"+str(i)+"?api_key="+api_key
        response = urlopen(url)
        data_json = json.loads(response.read())
        a={"id":i,
        "imdb_id":data_json["imdb_id"],
        "release_date":data_json["release_date"],
        "title":data_json["title"],
        "original_language":data_json["original_language"],
        "genres":data_json["genres"],
        "popularity":data_json["popularity"],
        "production_companies":data_json["production_companies"],
        "production_countries":data_json["production_countries"],
        "vote_average":data_json["vote_average"],
        "vote_count":data_json["vote_count"],
        "overview":data_json["overview"],
        "runtime":data_json["runtime"]
        }
        info.append(a)
    except:
        errors.append(i)
        
        

In [25]:
info=pd.DataFrame(info)

In [24]:
er=[]
movie_casts=[]
for i in info['id']:
    try:
        url="https://api.themoviedb.org/3/movie/"+str(i)+"/credits?api_key="+api_key+"&language=en-US"
        response = urlopen(url)
        data_json = json.loads(response.read())
        casts=[]
        for j in data_json['cast']:
            casts.append({
                'cast_id':j['id'],
                'name':j['name']
            })
        directors = [] 
        for j in data_json['crew']:
            if j["job"]=="Director":
                directors.append({
                    'cast_id':j['id'],
                    'name':j['name']
                })

        movie_casts.append({
            'id':i,
            'casts':casts,
            'directors':directors
        })
    except:
        print(i)
        er.append(i)
        

252983
570679
958475


In [28]:
casts.head()

,id,casts,directors
0,862,"[{'cast_id': 31, 'name': 'Tom Hanks'}, {'cast_...","[{'cast_id': 7879, 'name': 'John Lasseter'}]"
1,8844,"[{'cast_id': 2157, 'name': 'Robin Williams'}, ...","[{'cast_id': 4945, 'name': 'Joe Johnston'}]"
2,15602,"[{'cast_id': 6837, 'name': 'Walter Matthau'}, ...","[{'cast_id': 26502, 'name': 'Howard Deutch'}]"
3,31357,"[{'cast_id': 8851, 'name': 'Whitney Houston'},...","[{'cast_id': 2178, 'name': 'Forest Whitaker'}]"
4,11862,"[{'cast_id': 67773, 'name': 'Steve Martin'}, {...","[{'cast_id': 56106, 'name': 'Charles Shyer'}]"


In [32]:
len(info)

50141

we store the result in file

In [36]:
pd.merge(info, casts, how='inner', on = 'id').to_csv('data_movies.csv',index=False)